2QAN need to run under packages:

qiskit               0.36.2
qiskit-aer           0.10.4
qiskit-ibmq-provider 0.19.1
qiskit-ignis         0.7.1
qiskit-terra         0.20.2
py2QAN               0.0.3
retworkx             0.16.0
numpy                1.22.4

(https://github.com/unitaryfoundation/ucc-2qan)

You have two options: 1. you can use data in 'PhasePoly_benchmark' 
2. you can generate from networkx Graph.


In [2]:
(4203+1234+1313+406+116+34+69+22+116+643+189+527)/60

147.86666666666667

In [ ]:
# Import 2QAN compiler passes
 
# BenchArch defines the qubit architecture used in 2QAN
# and translates an OpenQASM circuit into Qiskit circuit.
from py2qan import BenchArch

# HeuristicMapper is the qubit mapping pass, 
# which finds an efficient mapping between circuit qubits and device qubits
# to reduce #SWAPs during qubit routing
from py2qan import HeuristicMapper

# QuRouter consists of a permutation-aware qubit routing pass 
# and a permutation-aware gate scheduling pass to minimise #SWAPs and circuit depth
from py2qan import QuRouter

In [ ]:
import os
import pickle as pkl
# Import qiskit 
import qiskit
from qiskit import transpile, QuantumCircuit

In [ ]:
import json
import numpy as np
# Read from JSON file
num_qubit = 16  # Length of each vector
k = 4  # Max True values allowed per vector
with open("PhasePoly_benchmark/MaxCut/"+"Regular"+ str(num_qubit) +"_"+ str(k) +".json", "r") as file:
    boolean_functions = json.load(file)

with open("PhasePoly_benchmark/MaxCut_layout/"+"Regular"+ str(num_qubit) +"_"+ str(k) +"_IBM_127.json", "r") as file:
    device_index_coupling_map = json.load(file)

In [ ]:
F = []
one_term_boolean_functions = []
for f in boolean_functions:
    if sum(f)==0:
        print("Incorrect! Containing all False boolean function")
        break
    if sum(f)==1:
        one_term_boolean_functions.append(f)
    if sum(f)>1:
        F.append(f)

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
def boolean_func_to_zz_qaoa_circuit(F, n_layer):
    """
    eg:
    # from util import evaluate_circuit
    # normal_qaoa, gamma, beta = boolean_func_to_normal_qaoa_circuit(F,1)
    # normal_qaoa = normal_qaoa.assign_parameters({gamma[0]: np.pi/5})
    # normal_qaoa = normal_qaoa.assign_parameters({beta[0]: np.pi/5})
    # normal_qaoa_prob = evaluate_circuit(normal_qaoa, 2**20)
    # normal_qaoa.draw()
    """
    theta = []
    for i in range(n_layer):
        theta.append(Parameter(f'γ_{i}'))
        theta.append(Parameter(f'β_{i}'))

    qc = QuantumCircuit(len(F[0]))
    
    for i in range(len(F[0])):
        qc.h(i)
    
    for l in range(n_layer):
        for row in F:
            true_indices = [i for i, val in enumerate(row) if val]
            for i in range(len(true_indices)-2):
                qc.cx(true_indices[i], true_indices[i+1])
            qc.rzz(np.pi/5,true_indices[len(true_indices)-2], true_indices[len(true_indices)-1])
            for i in range(len(true_indices)-2):
                qc.cx(true_indices[i], true_indices[i+1])
        for i in range(len(F[0])):
            qc.rx(np.pi/4,i)

    return qc, theta


zz_qc,params = boolean_func_to_zz_qaoa_circuit(F,1)

In [ ]:
def convert_to_sorted_index(pairs):
    # Get sorted unique values
    unique_values = sorted(set(num for pair in pairs for num in pair))

    # Create a mapping from original numbers to their sorted index
    index_map = {num: i for i, num in enumerate(unique_values)}

    # Replace numbers with their sorted index
    new_pairs = [[index_map[a], index_map[b]] for a, b in pairs]

    return new_pairs


# Convert to sorted index
coupling_map = convert_to_sorted_index(device_index_coupling_map)
num_physical_qubit = max(max(edge) for edge in coupling_map)+1

In [ ]:
mapper = 'qiskit' 
hmapper = HeuristicMapper(c_qasm, coupling_map=coupling_map) # The mapping pass in 2QAN
if mapper == 'qap':
    # The default mapper based on Quadratic Assignment Problem, very slow for #qubits > 30
    init_map, cost = hmapper.run_qap(num_iter=200, lst_len=20)
elif mapper == 'qiskit':
    # The SABRE mapper in Qiskit
    init_map = hmapper.run_qiskit(max_iterations=5)
    
print('The initial qubit map is \n', init_map) # keys are device qubits, values are circuit qubits

In [ ]:
import time
param = [[np.pi/5, np.pi/4]]
start_time = time.time()
layers = 1 # The number of QAOA layers
# Routing and scheduling, takes init_map as input
router = QuRouter(c_qasm, init_map=init_map, coupling_map=coupling_map) # The routing and scheduling passes in 2QAN
qaoa= True
if qaoa:
    # For QAOA, different layers have different gate parameters
    qs_circ, swaps1 = router.run_qaoa(layers=layers, gammas=param[layers-1][:layers], betas=param[layers-1][layers:], msmt=False) 
else:
    # For quantum simulation circuits, we assume each layer has the same time steps/parameters
    qs_circ, swaps1 = router.run(layers=layers, msmt='False')
    
# The routed and scheduled circuit: 
# all two-qubit gates are nearest-neighbouring gates on the device, but they have not been decompsed into native gate set
# Assuming the cx/cnot gate set in IBMQ
basis_gates = ['id', 'rz', 'cx', 'reset','h','rx']
decom_circ = transpile(qs_circ, basis_gates=basis_gates, optimization_level=3)
end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)

In [ ]:
# from qiskit_util import swap_to_cnot
from qiskit.converters import circuit_to_dag

def swap_rzz_to_cnot(qc):
    dag = circuit_to_dag(qc)
    new_qc = QuantumCircuit(qc.qubits, qc.clbits)
    for node in list(dag.topological_op_nodes()):
        qubits = node.qargs
        gate = node.op
        if gate.name == 'swap':
            new_qc.cx(qubits[0], qubits[1])
            new_qc.cx(qubits[1], qubits[0])
            new_qc.cx(qubits[0], qubits[1])
        elif gate.name == 'unitary':
            new_qc.cx(qubits[1], qubits[0])
            new_qc.cx(qubits[0], qubits[1])
            new_qc.rz(np.pi/5, qubits[0])
            new_qc.cx(qubits[1], qubits[0])
        elif gate.name == 'rzz':
            new_qc.cx(qubits[1], qubits[0])
            new_qc.rz(np.pi/5, qubits[0])
            new_qc.cx(qubits[1], qubits[0])
        else:
            new_qc.append(gate, qubits, node.cargs)
    return new_qc
no_swap_recovered_transpiled_bound_org_qc  = swap_rzz_to_cnot(qs_circ)

In [ ]:
from qiskit import QuantumCircuit, qpy
import json
with open("benchmark/MaxCut_Regular/2qan/"+"Regular"+ str(num_qubit) +"_"+ str(k) +"_2qan_circuit.qpy", "wb") as qpy_file_write:
    qpy.dump(no_swap_recovered_transpiled_bound_org_qc, qpy_file_write)